## EDA - Decision Tree for Titanic

**Import Package**

In [45]:
import pandas as pd

**Load Dataset**

In [46]:
train = pd.read_csv('input/train.csv', index_col = "PassengerId")    # train Dataset load
print(train.shape)
train.head()

(891, 11)


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [47]:
test = pd.read_csv('input/test.csv', index_col = "PassengerId")    # test Dataset load
print(test.shape)
test.head()

(418, 10)


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [48]:
submit = pd.read_csv('input/gender_submission.csv', index_col = "PassengerId")    # submission template load
print(submit.shape)
submit.head()

(418, 1)


,Survived
PassengerId,
892,0
893,1
894,0
895,0
896,1


## Preprocess
**Encode Sex**

In [49]:
train['Sex_Encoded'] = train['Sex'].replace('male', 0)\
                                   .replace('female', 1)

test['Sex_Encoded'] = test['Sex'].replace('male', 0)\
                                 .replace('female', 1)

In [50]:
# Check Encode Datd - Train
train[['Sex', 'Sex_Encoded']].head()

,Sex,Sex_Encoded
PassengerId,,
1,male,0
2,female,1
3,female,1
4,female,1
5,male,0


In [51]:
# Check Encode Data - Test
test[['Sex', 'Sex_Encoded']].head()

,Sex,Sex_Encoded
PassengerId,,
892,male,0
893,female,1
894,male,0
895,male,0
896,female,1


**Fill Missing Fare - Test -**

In [52]:
# Search Missing Fare - train
train.isnull().sum()

Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
Sex_Encoded      0
dtype: int64

In [53]:
test[test['Fare'].isnull()]

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_Encoded
PassengerId,,,,,,,,,,,
1044,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,NaN,S,0


In [54]:
# Search Missing Fare - test
test.isnull().sum()
# test.csv missing fare 1 fill 0
test['Fare'] = test['Fare'].fillna(0)
# Check missing fare
test.isnull().sum()

Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
Sex_Encoded      0
dtype: int64

**Encode Embarked**

In [64]:
# One Hot Encoding - over 2 data
# C == [True, False, False]
# S == [False, True, False]
# Q == [False, False, True]

# train Embarked encode
train['Embarked_C'] = train['Embarked'] == 'C'
train['Embarked_S'] = train['Embarked'] == 'S'
train['Embarked_Q'] = train['Embarked'] == 'Q'

print(train.shape)
train[['Embarked', 'Embarked_C', 'Embarked_S', 'Embarked_Q']].head()

(891, 15)


,Embarked,Embarked_C,Embarked_S,Embarked_Q
PassengerId,,,,
1,S,False,True,False
2,C,True,False,False
3,S,False,True,False
4,S,False,True,False
5,S,False,True,False


In [65]:
# test Embarked encode
test['Embarked_C'] = test['Embarked'] == 'C'
test['Embarked_S'] = test['Embarked'] == 'S'
test['Embarked_Q'] = test['Embarked'] == 'Q'

print(test.shape)
test[['Embarked', 'Embarked_C', 'Embarked_S', 'Embarked_Q']].head()

(418, 14)


,Embarked,Embarked_C,Embarked_S,Embarked_Q
PassengerId,,,,
892,Q,False,False,True
893,S,False,True,False
894,Q,False,False,True
895,S,False,True,False
896,S,False,True,False


## Decision Tree Training

**Feature setup**

In [66]:
# feature var-set
feature_names = ['Pclass', 'Sex_Encoded', 'Fare','Embarked_C', 'Embarked_S', 'Embarked_Q']
feature_names

['Pclass', 'Sex_Encoded', 'Fare', 'Embarked_C', 'Embarked_S', 'Embarked_Q']

In [67]:
# x-axis train feature
x_train = train[feature_names]
print(x_train.shape)
x_train.head()

(891, 6)


,Pclass,Sex_Encoded,Fare,Embarked_C,Embarked_S,Embarked_Q
PassengerId,,,,,,
1,3,0,7.2500,False,True,False
2,1,1,71.2833,True,False,False
3,3,1,7.9250,False,True,False
4,1,1,53.1000,False,True,False
5,3,0,8.0500,False,True,False


In [68]:
# x-axis test feature
x_test = test[feature_names]
print(x_test.shape)
x_test.head()

(418, 6)


,Pclass,Sex_Encoded,Fare,Embarked_C,Embarked_S,Embarked_Q
PassengerId,,,,,,
892,3,0,7.8292,False,False,True
893,3,1,7.0000,False,True,False
894,2,0,9.6875,False,False,True
895,3,0,8.6625,False,True,False
896,3,1,12.2875,False,True,False


**Label setup**

In [69]:
label_name = 'Survived'

y_train = train[label_name]
print(y_train.shape)
y_train.head()

(891,)


PassengerId
1    0
2    1
3    1
4    1
5    0
Name: Survived, dtype: int64

## Use Decision Tree
**import sklearn**

In [70]:
from sklearn.tree import DecisionTreeClassifier

# max depth limit
model = DecisionTreeClassifier(max_depth = 5)

**Fit & Predict**

In [71]:
# Fitting - Train
model.fit(x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=5, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [72]:
# Predict - Test
prediction = model.predict(x_test)
print(prediction.shape)
prediction[0:5]

(418,)


array([0, 0, 0, 0, 1], dtype=int64)

## Submission
**Input Submit Data**

In [73]:
submit['Survived'] = prediction

print(submit.shape)
submit.head()

(418, 1)


,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,1


**Make Submit File**

In [74]:
submit.to_csv('input/decisiontree02.csv')